In [1]:
import numpy as np
import pandas as pd

In [3]:
HousePrices = pd.read_csv("data/train.csv")

In [7]:
HousePrices.sample(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
1047,1048,20,RL,57.0,9245,Pave,NaN,IR2,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,145000
1216,1217,90,RM,68.0,8930,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,112000
1266,1267,190,RM,60.0,10120,Pave,NaN,IR1,Bnk,AllPub,...,0,NaN,MnPrv,NaN,0,1,2007,WD,Normal,122000
463,464,70,RL,74.0,11988,Pave,NaN,IR1,HLS,AllPub,...,0,NaN,NaN,NaN,0,8,2008,WD,Normal,188700
297,298,60,FV,66.0,7399,Pave,Pave,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2007,WD,Normal,239000


In [20]:
print(len(HousePrices.columns),
      '\n\n',
HousePrices.columns)

81 

 Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'Pave

In [16]:
HousePrices.isnull().sum(axis=0).sort_values(ascending = False).head(20)

PoolQC          1453
MiscFeature     1406
Alley           1369
Fence           1179
FireplaceQu      690
LotFrontage      259
GarageCond        81
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
BsmtExposure      38
BsmtFinType2      38
BsmtFinType1      37
BsmtCond          37
BsmtQual          37
MasVnrArea         8
MasVnrType         8
Electrical         1
Utilities          0
dtype: int64

In [71]:
mv = HousePrices.isnull().sum().sum()
dp = HousePrices.shape[0]*HousePrices.shape[1]
print(
    "\nRows:",
    len(HousePrices),
    "\nMissing values:",
    mv,
    "\ndatapoints:",
    dp,
    "\nMissingness:",
    str(round((mv/dp)*100,2)) +'%'
    
)


Rows: 1460 
Missing values: 6965 
datapoints: 118260 
Missingness: 5.89%


In [107]:
for col in HousePrices:
    print('=='*50)
    print(
        col,
        "\nData type:",
        HousePrices[[col]].dtypes[0],
        "\nNumber of Unique values:",
        HousePrices[[col]].nunique()[0],
          '\nMissing Values:',
          HousePrices[[col]].isnull().sum(axis=0)[0]
         )
    if HousePrices[[col]].nunique()[0] <= 10:
        print("Values:\n",
        HousePrices[col].unique())
    if HousePrices[[col]].nunique()[0] == 2:
        print('***BOOL***'*5)
    if (HousePrices[[col]].dtypes[0] == np.int64()) or (HousePrices[[col]].dtypes[0] == np.float64()):
        HousePrices[[col]] = HousePrices[[col]].dropna()
        print(
            "\nMax:",
            max(HousePrices[col]),
            "\nMin:",
            min(HousePrices[col]),
            "\naverage:",
            round(np.mean(HousePrices[col]),2),
            "\nStandard Deviation:",
            round(np.std(HousePrices[col]),2)
            
        )

Id 
Data type: int64 
Number of Unique values: 1460 
Missing Values: 0

Max: 1460 
Min: 1 
average: 730.5 
Standard Deviation: 421.47
MSSubClass 
Data type: int64 
Number of Unique values: 15 
Missing Values: 0

Max: 190 
Min: 20 
average: 56.9 
Standard Deviation: 42.29
MSZoning 
Data type: object 
Number of Unique values: 5 
Missing Values: 0
Values:
 ['RL' 'RM' 'C (all)' 'FV' 'RH']
LotFrontage 
Data type: float64 
Number of Unique values: 110 
Missing Values: 259

Max: 313.0 
Min: 21.0 
average: 70.05 
Standard Deviation: 24.27
LotArea 
Data type: int64 
Number of Unique values: 1073 
Missing Values: 0

Max: 215245 
Min: 1300 
average: 10516.83 
Standard Deviation: 9977.85
Street 
Data type: object 
Number of Unique values: 2 
Missing Values: 0
Values:
 ['Pave' 'Grvl']
***BOOL******BOOL******BOOL******BOOL******BOOL***
Alley 
Data type: object 
Number of Unique values: 2 
Missing Values: 1369
Values:
 [nan 'Grvl' 'Pave']
***BOOL******BOOL******BOOL******BOOL******BOOL***
LotShape 
D


Max: 2065 
Min: 0 
average: 346.99 
Standard Deviation: 436.38
LowQualFinSF 
Data type: int64 
Number of Unique values: 24 
Missing Values: 0

Max: 572 
Min: 0 
average: 5.84 
Standard Deviation: 48.61
GrLivArea 
Data type: int64 
Number of Unique values: 861 
Missing Values: 0

Max: 5642 
Min: 334 
average: 1515.46 
Standard Deviation: 525.3
BsmtFullBath 
Data type: int64 
Number of Unique values: 4 
Missing Values: 0
Values:
 [1 0 2 3]

Max: 3 
Min: 0 
average: 0.43 
Standard Deviation: 0.52
BsmtHalfBath 
Data type: int64 
Number of Unique values: 3 
Missing Values: 0
Values:
 [0 1 2]

Max: 2 
Min: 0 
average: 0.06 
Standard Deviation: 0.24
FullBath 
Data type: int64 
Number of Unique values: 4 
Missing Values: 0
Values:
 [2 1 3 0]

Max: 3 
Min: 0 
average: 1.57 
Standard Deviation: 0.55
HalfBath 
Data type: int64 
Number of Unique values: 3 
Missing Values: 0
Values:
 [1 0 2]

Max: 2 
Min: 0 
average: 0.38 
Standard Deviation: 0.5
BedroomAbvGr 
Data type: int64 
Number of Unique va